In [1]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

In [3]:
from langchain_anthropic import ChatAnthropic
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [4]:
db = SQLDatabase.from_uri("sqlite:///../../../../notebooks/Chinook.db")
llm = ChatAnthropic(
  model="claude-3-haiku-20240307",
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
)

OperationalError: (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)

- how to connect to the database?

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("How many employees are there?")